In [4]:
import numpy as np

In [5]:
class Golub_Kahan_SVD_Algorithm:
    def __init__(self,matrix):
        self.matrix = matrix
        self.num_rows, self.num_columns = self.GET_MATRIX_ROWS_COLUMNS(matrix) # Use a function to get rows and matrices
        self.computations = {} # Store Bidiag computations in a dict
        
    def GET_MATRIX_ROWS_COLUMNS(self, matrix):
        # Number of rows and columns in the matrix
        num_rows, num_cols = matrix.shape
        return num_rows, num_cols
    
    def SET_LOWVAL_ZERO(self, matrix):
        # This is for turning values into zeroes once they reach a certain threshold
        low_values_indices = abs(matrix) < 9e-15 
        matrix[low_values_indices] = 0
        return matrix
    
    def STORE_BIDIAG_COMPUTATIONS(self, i, x_vector, w_vector, v_vector, matrix): 
        # Store computations for the loop
        self.computations[i] = {
            'x_vector': x_vector.tolist(),
            'w_vector': w_vector.tolist(),
            'v_vector': v_vector.tolist(),
            'resulting_matrix': matrix.tolist()
        }
        return None
    
    def Householder_Reflector(self, vector, i):
        # For computing x, v, and w vectors as well as the P matrix
        # We change the sign depending on the sign of first element of the x vector
        alpha = -np.sign(vector[i]) * np.linalg.norm(vector)  
        e_vector = np.zeros(len(vector))
        e_vector[i] = 1.0
        
        # We then calculate the v and w vector as well as the P matrix
        w_vector = (vector - alpha * e_vector)
        v_vector = w_vector / np.linalg.norm(w_vector)
        P_matrix = np.eye(len(vector)) - 2 * np.outer(v_vector, v_vector.T)
        
        return P_matrix, vector, w_vector, v_vector
        
    def Golub_Kahan_Bidiagonalization(self):
        matrix = self.matrix
        
        # This algorithms will only run householder reflectors in the minimum no. of rows and columns
        # This is for cases of a non-square matrix A
        # Excess rows or columns will be left out
        if self.num_rows <= self.num_columns:
            num_iter = self.num_rows - 1
        else:
            num_iter = self.num_columns - 1
        
        for i in range(num_iter):
            # Performing Householder Reflectors column wise
            x_vector = np.zeros(len(matrix[:, i]))
            x_vector[i:] = matrix[i:, i]
            P_matrix, x_vector, w_vector, v_vector = self.Householder_Reflector(x_vector, i)
            matrix = self.SET_LOWVAL_ZERO(P_matrix @ matrix)
            self.STORE_BIDIAG_COMPUTATIONS(i, x_vector, w_vector, v_vector, matrix)

            # Performing Householder Reflectors row wise
            x_vector = np.zeros(len(matrix[i, :]))
            x_vector[i+1:] = matrix[i, i+1:] 
            Q_matrix, x_vector, w_vector, v_vector  = self.Householder_Reflector(x_vector, i+1)
            matrix = self.SET_LOWVAL_ZERO(matrix @ Q_matrix)
            self.STORE_BIDIAG_COMPUTATIONS(i+1, x_vector, w_vector, v_vector, matrix)
        
        # Truncate the resulting matrix
        matrix = np.trunc(matrix)   
        # Run the print function that prints out all of the computations for Bidiagonalization
        print_golub_kahan = self.print_household_reflector_computations()
        # Perform Tridiagonalization
        Tridiagonalization = self.Golub_Kahan_Tridiagonalization()
        return Tridiagonalization
    
    def Transform_to_square(self, B_matrix):
        # If in case the resulting bidiagonalized matrix is not square
        num_rows, num_columns = self.GET_MATRIX_ROWS_COLUMNS(B_matrix)
        
        # If there num of rows is greater than or equal to num of columns
        if num_rows >= num_columns:
            submatrix_C = B_matrix[0:num_columns, 0:num_columns]
            dim = num_columns
        else:
            # If num columns > num rows, get the number of excess columns
            add_cols = num_columns - num_rows
            if add_cols > 0:
                added_rows = np.zeros((add_cols, num_rows+add_cols))
            else:
                added_rows = np.zeros((num_rows+add_cols))
            submatrix_C = np.vstack((B_matrix[:num_rows, :num_rows+add_cols], added_rows))
            print(submatrix_C)
            dim = num_rows + add_cols
        
        return submatrix_C, dim
            
    
    def Golub_Kahan_Tridiagonalization(self):
        
        # Formatting options for np.array2string
        format_options = {
            'formatter': {'all': '{:.4f}'.format},  # Sspecific number of decimal places
            'suppress_small': True,  
            'separator': ', ',  
        }
        
        print("\033[1mPerforming tridiagonalization\033[0m")
        print("___________________________________________")
        print("")
       
        # Last computed bidiagonalized matrix
        last_iteration = list(self.computations.keys())[-1]
        B_matrix = self.computations[last_iteration]['resulting_matrix']
        matrix = np.array(B_matrix)

        # Reshape bidiagonalized matrix into a square
        B_matrix, dim = self.Transform_to_square(matrix)
        print(B_matrix)
        
        # Create matrix O full of zeroes
        O_matrix = np.zeros((dim, dim))
        
        # Create matrix M with blocking of O and B matrices
        M_matrix = np.block([[O_matrix,B_matrix.T],[B_matrix, O_matrix]])
        print("")
        print("\033[1mM matrix:\033[0m")
        print(np.array2string(M_matrix, **format_options))
        M_matrix_rows, M_matrix_cols = M_matrix.shape
        
        # Create Permutation Matrix
        P_matrix = np.zeros((M_matrix_cols, M_matrix_cols))
        
        # The algorithm for rearrnging Permutation matrix
        for i in range(dim):
            # Build permutation matrix depending on the dimensions of Matrix B
            P_matrix[i, i*2] = 1
            P_matrix[dim+i,2*i + 1] = 1
        
        print("")
        print("\033[1mP matrix:\033[0m")
        print(np.array2string(P_matrix, **format_options))
        
        # Computing for the tridiagonalized matrix by performing PᵀMP
        Resulting_matrix = P_matrix.T @ M_matrix @ P_matrix
        print("")
        print(print("\033[1mPᵀMP:\033[0m"))
        print(np.array2string(Resulting_matrix, **format_options))
        
        return Resulting_Matrix

    def print_household_reflector_computations(self):
        
        # For printing computations of Bidiagonalization that is stored from the dictionary
        # Formatting options for np.array2string
        format_options = {
            'formatter': {'all': '{:.4f}'.format},  # Specific number of decimal places
            'suppress_small': True,  
            'separator': ', ',  
        }

        print("________SOLUTION_________")
        print("")
        print("")
        
        # For each item in the dictionary we print x vector, w vector, v vector, and the resulting matrix    
        for i in range(len(self.computations)):
            iteration_data = self.computations[i]
            print("\033[1mIteration \033[0m" + str(i+1) + ":")

            print("\033[1mx vector: \033[0m" + str(iteration_data['x_vector']))
            print("\033[1mw vector: \033[0m" + str(iteration_data['w_vector']))
            print("\033[1mv vector: \033[0m" + str(iteration_data['v_vector']))
            print("")
            
            matrix = np.array(iteration_data['resulting_matrix'])

            # Printing resulting matrix each iteration
            print("\033[1mResulting matrix: \033[0m")
            print("")
            print(np.array2string(matrix, **format_options))
            print("")
            print("")


        return None
    
    def QR_Algorithm(self, T_matrix, tol=1e-8, max_iter = 1000):
        T_matrix = matrix
        n = T_matrix.shape[0]
        eigenvalues = np.zeros(n)
        iterations = 0
        
        while iterations < max_iter:
            print("T matrix =")
            print("          ")
            print(T_matrix)

            gram_schmidt = Gram_Schmidth_Process(T_matrix) # We call the Gram Schmidt class first
            gram_schmidt.compute_gram_schmidt() # We ask to make the computations
            gram_schmidt.display_formula_and_computations() # Then we ask it to display the computations

            Q_matrix = gram_schmidt.create_Q_matrix()  
            print("          ")
            print("Q matrix =") # Display the Q matrix
            print("          ")
            print(Q_matrix)

            R_matrix = gram_schmidt.create_R_matrix()
            print("          ")
            print("R matrix =") # Display the R matrix
            print("          ")
            print(R_matrix)
            
            T_matrix = R_matrix @ Q_matrix
            eigenvalues = np.diag(T_matrix)
            
            off_diagonal = np.sum(np.abs(T - np.diag(np.diag(T_matrix))))
            
            if off_diagonal < tol:
                break
        
            iterations += 1
            
        return eigenvalues



In [6]:
#We use np.random to generate a random matrix
A_matrix = np.random.rand(4, 4)

print("A matrix =")
print("          ")
print(A_matrix)

Solution = Golub_Kahan_SVD_Algorithm(A_matrix)
T_matrix = Solution.Golub_Kahan_Bidiagonalization()
Eigenvalues = T_matrix.QR_Algorithm(T_matrix)


A matrix =
          
[[0.83816679 0.36591573 0.95453094 0.24333989]
 [0.2934124  0.02715035 0.69154984 0.65375259]
 [0.99223828 0.46170891 0.6279889  0.38229135]
 [0.06383871 0.23725539 0.81848621 0.64738833]]
________SOLUTION_________


Iteration 1:
x vector: [0.8381667882745492, 0.29341239608094505, 0.9922382795396695, 0.06383871251744222]
w vector: [2.1712934078198054, 0.29341239608094505, 0.9922382795396695, 0.06383871251744222]
v vector: [0.9024196821388796, 0.121946264955891, 0.41238800320740776, 0.026532255129900203]

Resulting matrix: 

[[-1.3331, -0.5910, -1.2589, -0.6124],
 [0.0000, -0.1022, 0.3924, 0.5381],
 [0.0000, 0.0244, -0.3835, -0.0088],
 [0.0000, 0.2091, 0.7534, 0.6222]]


Iteration 2:
x vector: [0.0, -0.5022359777669363, 0.3117761173505079, -0.9562481971344186]
w vector: [0.0, -1.6264494535963687, 0.3117761173505079, -0.9562481971344186]
v vector: [0.0, -0.8505128787235157, 0.16303587086502075, -0.5000471457140043]

Resulting matrix: 

[[-1.3331, 1.5196, 0.0000, 0.0

NameError: name 'Resulting_Matrix' is not defined